In [1]:
import time
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re # regex
from IPython.display import Markdown as md
import datetime 
import json
import random
import csv
import openpyxl
import nltk
from nltk.corpus import opinion_lexicon
from textblob import TextBlob

In [2]:
fashionBrands = ['Nike','Zara',"Charles & Keith","Louis Vuitton", "Hermes", "Gucci", "Zalando", "Adidas", "H&M", "Cartier", "Lululemon", "Moncler", "Chanel",
                  "Prada", "Uniqlo", "Dior", "Forever 21", "ASOS", "Mango", "Versace", "Burberry", "Ralph Lauren", "Tommy Hilfiger",
                    "Calvin Klein", "Puma", "Under Armour", "Patagonia", "YSL"]

searchTerms = ["waste", "fast fashion", "sustainable", "consumerism", "quality", "animal testing", "price", "worth it",
                "haul", "aesthetic", "opinions", "thoughts", "comparison", "popular", "trends", "ethically sourced"]

years = [2019, 2020, 2021, 2022, 2023, 2024]

In [3]:
def soupify(link):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }
    soup_source = requests.get(link, headers=headers).text
    soup = BeautifulSoup(soup_source, 'html.parser')
    return soup

def response(link):
    response = requests.get(link)
    data = response.json()
    return data

def cit_url(link, driver):
    profile_URL = link
    # options = webdriver.ChromeOptions()
    # options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.9999.99 Safari/537.36")
    # driver = webdriver.Chrome(options=options)
    driver.get(profile_URL)
    time.sleep(1 + 2 * random.random())
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Brand', 'Search Term', 'Video Link', 'Comment'])
        writer.writerows(data)

In [4]:
options = webdriver.ChromeOptions()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.9999.99 Safari/537.36")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 5)
data=pd.DataFrame()

youtube_comments = []

for brand in fashionBrands:
    print(f"_____________________________________________{brand}")
    for search in searchTerms:
        print(search)
        query = f'{brand}+{search}+'
        url = 'https://www.youtube.com/results?search_query='+query

        try:
            soup = cit_url(url, driver)
            links = soup.find_all('a',id='video-title')[:5]
            for i in links:
                comment_count=0
                if 'shorts' in i['href']:
                    print(i['href'])
                else:
                    link = "https://www.youtube.com/"+i['href']
                    title = i.text.strip()
                    driver.get(link)

                    start_time = time.time()

                    while comment_count <= 52:
                        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)

                        # page_source = driver.page_source
                        # soup = BeautifulSoup(page_source, 'html.parser')
                        # likes = soup.find('div',class_='yt-spec-button-shape-next__button-text-content').text
                        # print(likes)

                        time.sleep(1)
                        comments = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content")))
                        #fixed_comments=[]

                        for comment in comments:
                            comment_text = comment.text.strip()  
                            if comment_text and len(comment_text.split()) > 1 and '\n' not in comment_text:
                                proper_comment = f'{title}+{comment_text}'
                                #fixed_comments.append(proper_comment)

                                entry = {
                                    'Brand':brand,
                                    'Search':search,
                                    'Comments': proper_comment,
                                    'Source': link,
                                    'Additional info': None
                                }

                                comment_count += 1
                                df_dictionary = pd.DataFrame([entry])
                                data = pd.concat([data, df_dictionary], ignore_index=True)

                        elapsed_time = time.time() - start_time
                        if elapsed_time > 10:
                            break

                    #youtube_comments.append([brand, search, '\n'.join(fixed_comments)])               

        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL: {url}")

data.to_csv("youtube.csv", index=False)
driver.quit()

_____________________________________________Nike
waste
fast fashion
sustainable
consumerism
quality
animal testing
price
worth it
haul
aesthetic
opinions
thoughts
comparison
popular
trends
ethically sourced
_____________________________________________Zara
waste
fast fashion
sustainable
consumerism
quality
animal testing
/shorts/luJRdgwY4dc
price
worth it
haul
aesthetic
opinions
thoughts
comparison
popular
trends
ethically sourced
_____________________________________________Charles & Keith
waste
fast fashion
sustainable
consumerism
quality
animal testing
price
worth it
haul
aesthetic
opinions
thoughts
comparison
popular
trends
ethically sourced
_____________________________________________Louis Vuitton
waste
fast fashion
sustainable
consumerism
quality
animal testing
price
worth it
haul
aesthetic
opinions
thoughts
comparison
popular
trends
ethically sourced
_____________________________________________Hermes
waste
fast fashion
sustainable
consumerism
quality
animal testing
price
wort

# Cleaning the scraped data

In [ ]:
data = pd.read_csv('youtube.csv')
data

,Brand,Search,Comments,Source,Additional info
0,Nike,waste,Space Hippie: These Nike Sneakers are Trash | ...,https://www.youtube.com//watch?v=Sm9r_Zs2z6s&p...,NaN
1,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
2,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
3,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
4,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
...,...,...,...,...,...
19947,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN
19948,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN
19949,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN
19950,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN


In [ ]:
nltk.download('opinion_lexicon')

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/shivangikrishna/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [ ]:
pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())

len(pos_list),len(neg_list)

(2006, 4783)

In [ ]:
'Great' in pos_list

False

In [ ]:
drop = []
rows_to_drop = []

for idx, row in data.iterrows():
    brand = row['Brand'].lower()
    main_comment = row['Comments'].lower()
    search = row['Search'].lower()
    comment = main_comment.split('+')[1]

    if not any(word in comment for word in pos_list) and not any(word in comment for word in neg_list) and brand not in comment and search not in comment:
        print(comment)
        drop.append(comment)
        rows_to_drop.append(idx)
    else:
        continue
data.drop(rows_to_drop, inplace=True)

0:00 / 2:39
0:00 / 2:27
muito bom
#movetozero  ️ ️ ️
never been a fashion forward person, but i want these shoes
my dream shoes
ficou show! ️
these behind the design videos are so clutch
yo quiero uno de esos !!!!
this looks
muito bom
#movetozero  ️ ️ ️
never been a fashion forward person, but i want these shoes
my dream shoes
ficou show! ️
these behind the design videos are so clutch
yo quiero uno de esos !!!!
this looks
make more
basketball shoes ?????
okay,  i'm sold!
i would go to space with these
fé. uugu
fé. uugu
fé. uugu
fé. uugu
fé. uugu
fé. uugu
fé. uugu
0:00 / 2:30
0:00 / 5:37
es una belleza!!!! me encantan
it's a piece of art
es una belleza!!!! me encantan
it's a piece of art
one day i will have one of these!
this handbag is on my wish-list
tôi thích thương hiệu này
es una belleza!!!! me encantan
it's a piece of art
one day i will have one of these!
this handbag is on my wish-list
tôi thích thương hiệu này
я ношу только такие чудесные сумочки для радости души, духа, тела; во

In [ ]:
len(drop)

2990

In [ ]:
data.drop_duplicates(inplace=True)
data

,Brand,Search,Comments,Source,Additional info
2,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
4,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
5,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
6,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
7,Nike,waste,From Trash to Space Hippie | Behind the Design...,https://www.youtube.com//watch?v=i3n_4-c1Rg8&p...,NaN
...,...,...,...,...,...
19943,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN
19947,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN
19948,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN
19949,YSL,waste,10 Reasons NOT to buy the Saint Laurent iCare ...,https://www.youtube.com//watch?v=fcZF_pHYH2c&p...,NaN


In [ ]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    
    if sentiment_score < 0:
        sentiment = "Negative"
    elif sentiment_score == 0:
        sentiment = "Neutral"
    else:
        sentiment = "Positive"
    
    return sentiment, sentiment_score

pos = 0
pos_list = []
neg = 0
neg_list = []
neutral = 0
neutral_list = []

for idx, row in data.iterrows():
    comment = row['Comments'].lower().split('+')[1]
    sentiment, sentiment_score = analyze_sentiment(comment)
    if sentiment == 'Positive':
        pos_list.append(comment)
        pos = pos+1
    elif sentiment == 'Negative':
        neg_list.append(comment)
        neg = neg+1
    else:
        neutral_list.append(comment)
        neutral = neutral+1

pos,neg,neutral

(5256, 1219, 1665)

In [ ]:
neutral_list

['i a nike é nota: 10 (dez)',
 'so this is all a lie?',
 'thank you for sharing the process!',
 "si vous me désigné comme vôtre seule représentant de la marque en algérie et en afrique et au moyen orient je peux vous aider à multiplié votre chiffre d'affaires par 100 en moins d'une année,️",
 'so fire',
 'good intentions, ugly shoes tho',
 'would have been nice if they hadn’t sold out in 30 seconds and that i could get a pair from sneakrs, when these drop in the us there’s going to be a lot of disappointment',
 'you cant get these shoes in the official nike store; currently they are getting sold in shoe resell sites such as goat and stockx for around 300 to 400 usd.',
 'what percent of the shoe is not made in a sweatshop?',
 'the name "space hippie" doesnt seem to fit well with the shoes. it doesnt seem to give justice to it!! step up your game, nike designers and innovators!!',
 'trash to cash',
 'if u use recycled material then how does every space hippie look uniform and same as the

In [ ]:
data.to_csv('cleaned_youtube_comments.csv')